<h1 style="text-align:center">Advanced Statistical Inference</h1>
<h1 style="text-align:center">Gaussian Process Regression</h1>

## 1. Aims
<div class="alert alert-info">
<ul> 
<li>To sample from a Gaussian process prior distribution.
<li>To implement Gaussian process inference for regression.
<li>To use the above to observe samples from a Gaussian process posterior distribution.
<li>To evaluate how different hyperparameter settings impact model quality.
<li> To investigate different kernel functions and parameter optimisation strategies.
</ul>
</div>

## 2. Introduction
<div class="alert alert-info">
Gaussian processes (henceforth GPs) achieve greater flexibility over parametric models by imposing a preference bias as opposed to restrictive constraints. Although the parameterisation of GPs allows one to access a certain (infinite) set of functions, preference can be expressed using a prior over functions. This allows greater freedom in representing data dependencies, thus enabling the construction of better-suited models. In this lab, we shall cover the basic concepts of GP regression. For the sake of clarity, we shall focus on univariate data, which allows for better visualisation of the GP model. Nonetheless, the code implemented within this lab can be very easily extended to handle
multi-dimensional inputs.
</div>

## 3. Sampling from the GP Prior
<div class="alert alert-info">
We shall consider a one-dimensional regression problem, whereby the inputs x are transformed by
a function $f(x) = sin(exp(0.03 * x))$. 
<br><br>
Generate 200 random points, $x$, in the range $[-20, 80]$, and compute their corresponding function
values, $t$. The target function can then be plotted accordingly.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
'''

TO COMPLETE

'''

<div class="alert alert-info">
Recall that since GPs are non-parametric; we define a prior distribution over functions (models),
specified as a multivariate Gaussian distribution $p(f) = N (\mu, \Sigma)$.

Without loss of generality, we shall assume a zero-mean GP prior, i.e. $\mu = 0$. The covariance
matrix of the distribution, $\Sigma$, may then be computed by evaluating the covariance between the
input points. For this tutorial, we shall consider the widely used squared-exponential (RBF)
covariance (also referred to as the kernel function), which is defined between two points as: 

$$k(x, x') = \sigma_f^2 \exp \Big( -\dfrac {(x-x')^2}{2l^2} \Big). $$

This kernel is parameterised by a lengthscale parameter $l$, and variance $\sigma_f^2$ . Given that the true
function may be assumed to be corrupted with noise, we can also add a noise parameter, $\sigma_n^2$ , to
the diagonal entries of the resulting kernel matrix, $K$, such that

$$K_y = K + \sigma_n^2I.$$
<br>
Complete the `compute_kernel()` function for computing the RBF kernel $K$ between two sets of input points.<br><br>

Hint: The ‘cdist’ function in scipy can be used for evaluating the pairwise Euclidean distance between two sets of points.
</div>


In [ ]:
def compute_kernel(x1, x2, lengthscale, variance):

    '''
    
    TO COMPLETE
    
    '''

<div class="alert alert-info">
Assuming a zero-mean prior, and using the kernel matrix constructed with `compute_kernel()` for input points x, we can sample from the prior distribution using the numpy `multivariate_normal()` function.<br><br>

For the time being, you can initialise the kernel parameters as follows:
<br>
- lengthscale = 10<br>
- variance = 0.1
</div>

In [ ]:
mu = ... # TO COMPLETE
K = ... # TO COMPLETE
R = np.random.multivariate_normal(...) # TO COMPLETE

<div class="alert alert-info">
On the same figure where we plotted the true function, draw four sample functions from the prior distribution and plot them accordingly.
</div>

In [ ]:
'''

TO COMPLETE

'''

<div class="alert alert-info">
 In order to better understand the role of the hyperparameters, observe how altering the kernel parameters impacts the shape of the prior samples.</div>

In [ ]:
'''

TO COMPLETE

'''

## 4. GP Inference
<div class="alert alert-info">
Recall that the prior represents our prior beliefs before observing the function values of any data points. Suppose we can now observe 3 points at random from the input data; we would expect that with this additional knowledge, the functions drawn from the updated GP distribution would be constrained to pass through these points (or at least close if corrupted with noise). The combination of the prior and the likelihood of the observed data leads to the posterior distribution over functions.
<br><br>
Assign 3 points at random from $x$ (and their corresponding function values) to `obs_x` and `obs_t`
respectively. For now we shall assume that all other $x$ values are unobserved.<br><br>

You are encouraged to use the given initial configuration.
</div>

In [ ]:
lengthscale = 3
variance = 2
noise = 1e-3

params = [lengthscale, variance, noise]

In [ ]:
numObs = 10
obs_x = np.random.choice(x, numObs, replace=False)
obs_t = ... # TO COMPLETE

<div class="alert alert-info">

Complete the provided implementation of `gp_inference` for evaluating the posterior GP mean and variance using the equations given in the lecture.
<br><br>
<b>Note</b>: As we have encountered in previous labs, matrix inversions can be both numerically troublesome and slow to compute. In this lab, we shall avoid computing matrix inversions directly by instead considering Cholesky decompositions for solving linear systems. You are encouraged to read more about Cholesky decompositions for GPs by consulting Appendix A.4 of <a target="_blank" href="http://www.gaussianprocess.org/gpml/">Gaussian Processes for Machine Learning (Rasmussen and Williams, 2005)</a> - available online!<br><br>

The complete pseudo-code for the following procedure is provided in Algorithm 2.1 from Chapter 2 of this same book.
</div>

In [ ]:
def gp_inference(obs_x, obs_t, x_new, params):
    
    # unpack params
    lengthscale = params[0]
    variance = params[1]
    noise = params[2]
    
    N = np.shape(obs_x)[0]
    
    # compute kernel
    K = ... 
    K_y = ...
    
    '''
    
    When computing the posterior mean, we would like to avoid evaluating
        
                            alpha = np.linalg.inv(K_y).dot(obs_t) 
    
    directly. The Cholesky decomposition can be applied using the following procedure.
    
        -> Compute the lower triangular Cholesky decomposition of K_y (which we shall call K_chol)
        -> Compute 'alpha' as:
        
                            alpha = K_chol.T \ (K_chol \ obs_t)
                            
           where the back-substitution operator can be evaluated using the 'solve_triangular' 
           function in scipy. Make sure to set the function's lower' flag as appropriate.
            
    
    '''
    
    
    # compute the Cholesky decomposition of K_y
    K_chol = ...
    
    # compute alpha
    alpha = ...
    
    # compute the covariance between the training and test data
    K_obs_pred = ...
    
    # compute the covariance for the test data
    K_pred = ...
    
    # compute the posterior mean
    posterior_m = np.dot(K_obs_pred.T, alpha)
    
    
    '''
    
    Similarly, when computing
    
                        v = np.linalg.inv(kern_obs)*kern_obs_pred
                        
    employ the Cholesky decomposition as outlined above.
                        
    '''
    
    v = ...
    posterior_v = ...
    
    
    return posterior_m, posterior_v

## 5. Sampling from the  GP Posterior<br>

<div class="alert alert-info">
Now that you have computed the posterior mean and variance, create a new figure once again showing the true function. To this figure, add the posterior mean and uncertainty (show two standard deviations) evaluated on the same $x$ values.
</div>

In [ ]:
posterior_m, posterior_v = gp_inference(...) # TO COMPLETE
R2 = np.random.multivariate_normal(...) # TO COMPLETE


'''

The variance at each point is given by the diagonal of the variance matrix.
Recall that the standard deviation is the square root of the variance.

'''

posterior_std = ... # TO COMPLETE


In [ ]:
'''

Plot the results using error bars in order to display the posterior uncertainty.

TO COMPLETE

'''


<div class="alert alert-info">

Note that we should also add the noise variance to the predictive variance of the posterior. Fix this accordingly.
</div>

In [ ]:
'''

TO COMPLETE

'''

<div class="alert alert-info">

Additionally, as we did with the prior distribution, sample four functions from the posterior and plot them on the same figure. Comment on the obtained plots. You should also view how the posterior mean and variance change when more observations are treated as observations in the GP inference procedure.
</div>

In [ ]:
'''

TO COMPLETE

'''

<div class="alert alert-info">
As a measure of model quality, you should also compute the log marginal likelihood of the model.
To this end, complete the code provided in `gp inference()` to include the negative log likelihood term.
<br><br>
You could attempt a grid search over a range of parameter values in order to determine which configuration yields the best result.<br><br>

Hint: Once again, refer to Algorithm 2.1 in Chapter 2 of the book referenced above for details on how to compute this term
efficiently.
</div>

In [ ]:
'''

TO COMPLETE

'''

## Bonus: Alternative Kernel Functions
<div class="alert alert-info">
In this notebook, we have focused exclusively on the RBF kernel.
However, the choice of kernel function (along with its associated parameters) can have a significant effect on the overall Gaussian process model.
Choosing the best kernel to fit your data is no simple task, and is a pertinent problem in many applied domains.<br>

A brief discussion on this problem may be found here: <a target="_blank" href="https://www.cs.toronto.edu/~duvenaud/cookbook/">Kernel Cookbook</a>. 

Familiarise yourself better with these issues by implementing one or two additional kernels.
The effectiveness of each kernel may then be easily evaluated by simply overriding the previously defined `compute_kernel()` function.
As a further step, you could also extend this comparison to other test functions besides the one being used in this exercise.

In [ ]:
'''

TO COMPLETE

'''

## Bonus: Parameter Optimisation using Gradient Descent
<br>
<div class="alert alert-info">
Optimise the hyperparameters of the model by minimising the negative log-likelihood of the model. For a complete solution, you should include the derivatives of the objective function with respect to the parameters being optimised.
<br><br>
The general formula for computing the derivative is given below:<br>
$$
\frac{\partial\;\text{NLL}}{\partial\;\theta_i} = - \frac{1}{2} \textbf{Tr} \left( K^{-1} \frac{\partial K}{\partial \theta_i} \right) + \frac{1}{2} \textbf{y}^{T} K^{-1} \frac{\partial K}{\partial \theta_i} K^{-1} \textbf{y}.
$$<br>
To give a more concrete example, the $\frac{\partial K}{\partial \theta_i}$ term for the lengthscale parameter in the RBF kernel is computed as follows:
$$
\frac{\partial K}{\partial l} = \sigma_f^2 \exp \left( -\dfrac {(x-x')^2}{2l^2} \right)\left( \dfrac {(x-x')^2}{l^3} \right)
$$
<br><br>
<b>Pro tip:</b> Note that the parameters $l$, $\sigma_f^2$ , and $\sigma_n^2$ are always expected to be positive. It is possible that the optimisation algorithm attempts to evaluate the log-likelihood in regions of the parameter space where one or more of these parameters are negative, leading to numerical issues. While not expected in this notebook, a commonly-used technique to enforce this condition is to work with a transformed version of covariance parameters using the logarithm transformation. In particular, define $\psi_l = log(l)$, $\psi_f = log(\sigma_f^2 )$, and $\psi_n = log(\sigma_n^2 )$, and optimise with respect to the $\psi$ parameters. The optimisation problem in the transformed space is now unbounded, and the gradient of the log-likelihood should be computed with respect to the $\psi$ parameters.
</div>

In [ ]:
'''

TO COMPLETE

'''